In [1]:
import scipy
import logging
import atlite
import numpy as np
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
import logging

In [3]:
from init import *

In [4]:
from src.optimize import optimize_pixel_by_coord
from src.logging_config import setup_logging

In [5]:
setup_logging()
logging.getLogger().setLevel(logging.DEBUG)

[2023-08-08 14:47:28,831] INFO  - root : Starting /home/pregner/micromamba/envs/syfop-global-costs/lib/python3.9/site-packages/ipykernel_launcher.py....


In [6]:
logging.getLogger().setLevel(logging.DEBUG)

In [7]:
import os
os.environ['GRB_LICENSE_FILE'] = '/home/pregner/gurobi.lic'

# this does not work in jupyter notebook, needs to be set in kernel spec!
# see /home/pregner/micromamba/envs/syfop-global-costs/share/jupyter/kernels/python3/kernel.json
# os.environ['LD_LIBRARY_PATH'] = '/opt/gurobi1001/linux64/lib'

In [8]:
# this is not really needed if we have a complete solution and concat script is executed

import glob
fnames = glob.glob(str(INTERIM_DIR / "network_solution"  / "*.nc"))
solution = xr.open_mfdataset(fnames)

In [9]:
solution = solution.load()

In [10]:
# is x/y correct?
x, y  = np.array(np.where(solution.runtime_solver > 250))[:, 0] + np.array([config.X_IDX_FROM_TO[0], config.Y_IDX_FROM_TO[0]])

In [11]:
x - config.X_IDX_FROM_TO[0]

0

In [12]:
y - config.Y_IDX_FROM_TO[0]

12

In [13]:
%%time

optimize_pixel_by_coord(
    x=x+3,
    y=y+1,
    #model_file='/tmp/model_0_9.lp'
    **{
        #'simplex.perturbation.constant': 1e-6,
        #'simplex.perturbatjion.indicator': True,
        #'basis_fn': '/tmp/solution.sol',
        #'warmstart_fn': '/tmp/solution.sol',
    },
)

[2023-08-08 14:47:57,329] INFO  - root : Load renewable time series...


Automatic pdb calling has been turned ON


[2023-08-08 14:47:57,534] INFO  - root : Start optimization...
[2023-08-08 14:47:58,080] INFO  - root : Creating network took 0.5451996326446533
[2023-08-08 14:47:58,081] INFO  - linopy.model:  Solve linear problem using Gurobi solver
[2023-08-08 14:47:58,081] INFO  - linopy.model: Solver options:
 - BarHomogeneous: 1
 - ScaleFlag: 0
 - Method: 2
 - Aggregate: 2
 - AggFill: 0
 - PrePasses: 8
[2023-08-08 14:53:04,926] INFO  - linopy.constants:  Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 183969 primals, 254040 duals
Objective: 4.21e+04
Solver model: available
Solver message: 2

[2023-08-08 14:53:05,014] INFO  - root : Solving time: 306.9334237575531
[2023-08-08 14:53:05,015] DEBUG - root : Solver output: 

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-02
Set parameter BarHomogeneous to value 1
Set parameter ScaleFlag to value 0
Set parameter Method to value 2
Set parameter Aggregate to value 2
Set parameter AggF

(<xarray.Dataset>
 Dimensions:                                  (x: 1, y: 1, time: 8760)
 Coordinates:
   * x                                        (x) float64 5.75
   * y                                        (y) float64 53.25
   * time                                     (time) datetime64[ns] 2011-01-01...
 Data variables: (12/32)
     size_solar_pv                            (x, y) float64 0.6071
     size_wind                                (x, y) float64 0.4937
     flow_solar_pv_electricity                (x, y, time) float64 0.0 ... 0.0
     flow_wind_electricity                    (x, y, time) float64 0.4091 ... ...
     size_storage_electricity                 (x, y) float64 0.0
     storage_level_electricity                (x, y, time) float64 0.0 ... 0.0
     ...                                       ...
     storage_discharge_methanol_synthesis     (x, y, time) float64 0.0 ... 999.8
     size_methanol_synthesis                  (x, y) float64 1e+03
     flow_methanol_synt

In [18]:
model = _13[1].model

In [20]:
model_highs = model.to_highspy()

Running HiGHS 1.5.0 [date: 2022-12-15, git hash: fb38df1ce]
Copyright (c) 2022 HiGHS under MIT licence terms


In [22]:
model_highs.readSolution(

Docstring: readSolution(self: highspy.highs_bindings._Highs, arg0: str, arg1: int) -> highspy.highs_bindings.HighsStatus
Type:      method

In [24]:
%pdb

Automatic pdb calling has been turned OFF


In [33]:
model_highs.readSolution(str(INTERIM_DIR / 'solution.sol'))

TypeError: readSolution(): incompatible function arguments. The following argument types are supported:
    1. (self: highspy.highs_bindings._Highs, arg0: str, arg1: int) -> highspy.highs_bindings.HighsStatus

Invoked with: <highspy.highs.Highs object at 0x7fd943dca540>, '/data/users/pregner/syfop-global-costs/notebooks/../data/interim/solution.sol'

In [74]:
%%time

optimize_pixel_by_coord(
    x=x+3,
    y=y+1,
    #model_file='/tmp/model_0_9.lp'
    **{
        'solver_name': 'highs',
        #'threads': 1,
        #'lpmethod': 4,
        #'barrier.startalg': 2,
        #'solutiontype': 2,
        
        #'simplex.perturbation.constant': 1e-6,
        #'simplex.perturbation.indicator': True,
        'basis_fn': '/tmp/solution.sol',
        #'warmstart_fn': '/tmp/solution.sol',
    },
)

INFO:root:Load renewable time series...
INFO:root:Start optimization...
INFO:root:Creating network took 0.5293958187103271
INFO:linopy.model: Solve linear problem using Highs solver
INFO:linopy.model:Solver options:
 - simplex.perturbation.constant: 1e-06
 - simplex.perturbation.indicator: True
 - threads: 1
 - lpmethod: 4
 - barrier.startalg: 2


Running HiGHS 1.5.0 [date: 2022-12-15, git hash: fb38df1ce]
Copyright (c) 2022 HiGHS under MIT licence terms


INFO:linopy.solvers:Log file at /tmp/highs.log.
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 183969 primals, 254040 duals
Objective: 4.22e+04
Solver model: available
Solver message: optimal

INFO:root:Solving time: 564.2230291366577
DEBUG:root:Solver output: 


INFO:root:Solver runtime: nan


ERROR:   getOptionIndex: Option "simplex.perturbation.constant" is unknown
ERROR:   getOptionIndex: Option "simplex.perturbation.indicator" is unknown
ERROR:   getOptionIndex: Option "lpmethod" is unknown
ERROR:   getOptionIndex: Option "barrier.startalg" is unknown
Presolving model
175200 rows, 140168 cols, 519421 nonzeros
155893 rows, 112102 cols, 465870 nonzeros
155863 rows, 112071 cols, 465808 nonzeros
Presolve : Reductions: rows 155863(-98177); columns 112071(-71898); elements 465808(-228813)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2(23.4375) 0s
       6860     4.2000517912e+04 Pr: 4(8.65832); Du: 0(8.68776e-07) 5s
      10518     4.2000517920e+04 Pr: 4(7.52418); Du: 0(4.59149e-06) 10s
      14256     4.2000517928e+04 Pr: 4(8.65832); Du: 0(4.97348e-07) 15s
      18236     4.2007500916e+04 Pr: 32220(4.96831) 21s
      20618     4.2008911761e+04 Pr: 26762(4.96935) 2

(<xarray.Dataset>
 Dimensions:                                  (x: 1, y: 1, time: 8760)
 Coordinates:
   * x                                        (x) float64 5.75
   * y                                        (y) float64 52.5
   * time                                     (time) datetime64[ns] 2011-01-01...
 Data variables: (12/32)
     size_solar_pv                            (x, y) float64 1.112
     size_wind                                (x, y) float64 0.4697
     flow_solar_pv_electricity                (x, y, time) float64 -0.0 ... -0.0
     flow_wind_electricity                    (x, y, time) float64 0.143 ... 0...
     size_storage_electricity                 (x, y) float64 -0.0
     storage_level_electricity                (x, y, time) float64 -0.0 ... 0.0
     ...                                       ...
     storage_discharge_methanol_synthesis     (x, y, time) float64 0.0 ... 999.8
     size_methanol_synthesis                  (x, y) float64 1e+03
     flow_methanol_sy

In [15]:
%%time

optimize_pixel_by_coord(
    x=x+3,
    y=y+3,
    #model_file='/tmp/model_0_9.lp'
    **{
        'solver_name': 'gurobi',
        #'threads': 1,
        #'lpmethod': 4,
        #'barrier.startalg': 2,
        #'solutiontype': 2,
        
        #'simplex.perturbation.constant': 1e-6,
        #'simplex.perturbation.indicator': True,
        #'basis_fn': '/tmp/solution.sol',
        #'warmstart_fn': '/tmp/solution.sol',
    },
)

[2023-08-08 11:45:11,138] INFO  - root : Load renewable time series...
[2023-08-08 11:45:11,341] INFO  - root : Start optimization...
[2023-08-08 11:45:11,875] INFO  - root : Creating network took 0.5322911739349365
[2023-08-08 11:45:11,876] INFO  - linopy.model:  Solve linear problem using Gurobi solver
[2023-08-08 11:45:11,876] INFO  - linopy.model: Solver options:
 - simplex.perturbation.constant: 1e-06
 - simplex.perturbation.indicator: True
 - threads: 1
 - lpmethod: 4
 - barrier.startalg: 2
[2023-08-08 11:45:24,860] WARNING - linopy.constants: Optimization failed: 
Status: aborted
Termination condition: user_interrupt
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: 11




Interrupt request received


SolverError: unable to solve optimization, solver status=aborted, termination condition=user_interrupt

In [10]:
import highspy
h = highspy.Highs()

In [52]:
%%time

optimize_pixel_by_coord(
    x=x+3,
    y=y+1,
    #model_file='/tmp/model_0_9.lp'
    **{
        #'threads': 1,
        #'lpmethod': 4,
        #'barrier.startalg': 2,
        #'solutiontype': 2,
        
        #'simplex.perturbation.constant': 1e-6,
        #'simplex.perturbation.indicator': True,
        #'basis_fn': '/tmp/solution.sol',
        #'warmstart_fn': '/tmp/solution.sol',
    },
)

INFO:root:Load renewable time series...
INFO:root:Start optimization...
INFO:root:Creating network took 2.466878890991211
INFO:linopy.model: Solve linear problem using Cplex solver
INFO:linopy.model:Solver options:
 - simplex.perturbation.constant: 1e-06
 - simplex.perturbation.indicator: True
 - threads: 1
 - lpmethod: 4
 - barrier.startalg: 2
INFO:linopy.io:Writing objective.
Writing variables.: 100%|██████████| 30/30 [00:00<00:00, 71.40it/s]
INFO:linopy.io: Writing time: 3.29s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 183969 primals, 254040 duals
Objective: 4.22e+04
Solver model: available
Solver message: optimal

INFO:root:Solving time: 115.13986396789551
DEBUG:root:Solver output: 

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Simplex_Perturbation_Indicator          1
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                4
CPXPARAM_Threads                      

CPU times: user 1min 56s, sys: 2.02 s, total: 1min 58s
Wall time: 1min 58s


(<xarray.Dataset>
 Dimensions:                                  (x: 1, y: 1, time: 8760)
 Coordinates:
   * x                                        (x) float64 5.75
   * y                                        (y) float64 52.5
   * time                                     (time) datetime64[ns] 2011-01-01...
 Data variables: (12/32)
     size_solar_pv                            (x, y) float64 1.112
     size_wind                                (x, y) float64 0.4697
     flow_solar_pv_electricity                (x, y, time) float64 0.0 ... 0.0
     flow_wind_electricity                    (x, y, time) float64 0.143 ... 0...
     size_storage_electricity                 (x, y) float64 0.0
     storage_level_electricity                (x, y, time) float64 0.0 ... 0.0
     ...                                       ...
     storage_discharge_methanol_synthesis     (x, y, time) float64 0.0 ... 999.8
     size_methanol_synthesis                  (x, y) float64 1e+03
     flow_methanol_synthe

In [54]:
solution_basic = _52[0].copy()

In [ ]:
solution_nonbasic = _42[0].copy()

In [50]:
solution_nonbasic

<xarray.Dataset>
Dimensions:                                  (x: 1, y: 1, time: 8760)
Coordinates:
  * x                                        (x) float64 5.75
  * y                                        (y) float64 52.5
  * time                                     (time) datetime64[ns] 2011-01-01...
Data variables: (12/32)
    size_solar_pv                            (x, y) float64 1.11
    size_wind                                (x, y) float64 0.4703
    flow_solar_pv_electricity                (x, y, time) float64 0.0 ... 0.0
    flow_wind_electricity                    (x, y, time) float64 0.1432 ... ...
    size_storage_electricity                 (x, y) float64 1.214e-05
    storage_level_electricity                (x, y, time) float64 4.133e-06 ....
    ...                                       ...
    storage_discharge_methanol_synthesis     (x, y, time) float64 0.0 ... 999.9
    size_methanol_synthesis                  (x, y) float64 1e+03
    flow_methanol_synthesis_methanol_demand  (x, y, time) float64 0.0 ... 1e+03
    output_flow_curtail_electricity          (x, y, time) float64 9.063e-08 ....
    runtime_solver                           (x, y) float64 6.8
    runtime                                  (x, y) float64 9.178

In [58]:
solution_nonbasic.size_storage_electricity

<xarray.DataArray 'size_storage_electricity' (x: 1, y: 1)>
array([[1.213723e-05]])
Coordinates:
  * x        (x) float64 5.75
  * y        (y) float64 52.5

In [59]:
solution_basic.size_storage_electricity

<xarray.DataArray 'size_storage_electricity' (x: 1, y: 1)>
array([[0.]])
Coordinates:
  * x        (x) float64 5.75
  * y        (y) float64 52.5

In [61]:
solution_nonbasic.storage_level_co2.min()

<xarray.DataArray 'storage_level_co2' ()>
array(2.71270824e-06)

In [62]:
solution_basic.storage_level_co2.min()

<xarray.DataArray 'storage_level_co2' ()>
array(0.)

In [64]:
(solution_basic - solution_nonbasic).max()

<xarray.Dataset>
Dimensions:                                  ()
Data variables: (12/32)
    size_solar_pv                            float64 0.001947
    size_wind                                float64 -0.0005985
    flow_solar_pv_electricity                float64 0.001567
    flow_wind_electricity                    float64 0.0
    size_storage_electricity                 float64 -1.214e-05
    storage_level_electricity                float64 -9.589e-08
    ...                                       ...
    storage_discharge_methanol_synthesis     float64 0.0
    size_methanol_synthesis                  float64 -1.779e-10
    flow_methanol_synthesis_methanol_demand  float64 0.0
    output_flow_curtail_electricity          float64 0.5183
    runtime_solver                           float64 102.3
    runtime                                  float64 108.4